In [1]:
pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=01299ba89d4bd2f328b588c79c197d3c78454eaf14157f0ed6972d9ae9e68748
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [2]:
import os
import torch
import torchvision
from sklearn.metrics import classification_report, accuracy_score
from torchvision import transforms
from PIL import Image
from efficientnet_pytorch import EfficientNet

In [3]:
# Define transformations
transform_rotation = transforms.RandomApply([
    transforms.RandomRotation(20)
], p=0.2)

transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.2),
    transform_rotation,
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# DataLoaders
TRAIN_DATA_DIR = r'/content/drive/MyDrive/catEmotion/catEmotionTrain'
TEST_DATA_DIR = r'/content/drive/MyDrive/catEmotion/catEmotionTest'

BATCH_SIZE = 32

train_data = torchvision.datasets.ImageFolder(TRAIN_DATA_DIR,
                                               transform=transform_train,
                                               is_valid_file=lambda x: x.endswith('.jpg'))

test_data = torchvision.datasets.ImageFolder(TEST_DATA_DIR,
                                             transform=transform_test,
                                             is_valid_file=lambda x: x.endswith('.jpg'))

train_data_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
)

test_data_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
)

In [6]:
# Model
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=4)


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 140MB/s]

Loaded pretrained weights for efficientnet-b0


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}.')


Using device: cpu.


In [8]:
import os

MODEL_SAVE_PATH = 'checkpoints'
if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH)

# Now you can proceed with the training code


In [9]:
# Training
MODEL_SAVE_PATH = 'checkpoints'
LEARNING_RATE = 1e-3
N_EPOCHS = 5

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

def train(model, criterion, optimizer, train_data_loader, device, model_save_path, n_epochs=5):
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_data_loader.dataset)
        print(f"Epoch {epoch+1}/{n_epochs} Loss: {epoch_loss:.4f}")
        torch.save(model.state_dict(), os.path.join(model_save_path, f'epoch_{epoch + 1}_checkpoint.pth'))



train(model, criterion, optimizer, train_data_loader, device, MODEL_SAVE_PATH)




Epoch 1/5 Loss: 0.8006
Epoch 2/5 Loss: 0.1627
Epoch 3/5 Loss: 0.1078
Epoch 4/5 Loss: 0.0653
Epoch 5/5 Loss: 0.0824


In [10]:
# Testing
model.load_state_dict(torch.load(os.path.join(MODEL_SAVE_PATH, 'epoch_5_checkpoint.pth')))
model.eval()

y_true = []
y_pred = []
for test_data, test_labels in test_data_loader:
    test_data = test_data.to(device)
    test_labels = test_labels.to(device)
    with torch.no_grad():
        test_preds = model(test_data)
    test_pred_labels = torch.argmax(test_preds, dim=1)
    y_true.extend(test_labels.detach().cpu().numpy())
    y_pred.extend(test_pred_labels.detach().cpu().numpy())


In [11]:
# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Print accuracy and classification report
print(f'Accuracy: {accuracy}')
print(classification_report(y_true, y_pred))

Accuracy: 0.8952380952380953
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        76
           1       0.96      0.86      0.91        51
           2       0.97      0.94      0.95        32
           3       1.00      0.75      0.85        51

    accuracy                           0.90       210
   macro avg       0.93      0.89      0.90       210
weighted avg       0.91      0.90      0.89       210



In [12]:
from google.colab import files

# Provide the path to the checkpoint file
checkpoint_path = '/content/checkpoints/epoch_5_checkpoint.pth'

# Download the checkpoint file
files.download(checkpoint_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
from google.colab import files

# Provide the path to the checkpoint file
checkpoint_path = '/content/checkpoints/epoch_4_checkpoint.pth'

# Download the checkpoint file
files.download(checkpoint_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from google.colab import files

# Provide the path to the checkpoint file
checkpoint_path = '/content/checkpoints/epoch_3_checkpoint.pth'

# Download the checkpoint file
files.download(checkpoint_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
from google.colab import files

# Provide the path to the checkpoint file
checkpoint_path = '/content/checkpoints/epoch_2_checkpoint.pth'

# Download the checkpoint file
files.download(checkpoint_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
from google.colab import files

# Provide the path to the checkpoint file
checkpoint_path = '/content/checkpoints/epoch_1_checkpoint.pth'

# Download the checkpoint file
files.download(checkpoint_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>